<a href="https://colab.research.google.com/github/luaiabuelsamen/McHacks9-TMIL/blob/master/NLP_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cryptocurreny Market Optimism analyser

1. Import libraries and dependencies

In [73]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch import softmax
import numpy as np
import pandas as pd

2. Load pre-trained multilingual sentiment (positivity) prediction model


In [62]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

3. Tweets import and pre-processing

In [132]:
!pip install tweepy --upgrade

     |████████████████████████████████| 65 kB 2.6 MB/s 
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0


In [17]:
import tweepy #ver 4.4.0
#Twitter API stuff
consumerKey = "2xahjEG9TT3v3lV2kFCdFAEjt"
consumerSecret = "LuYQEXzpBRl0J3EIAYeB9t61rt7wyhMnvgFHJtWKbHUG8uib3m"
accessToken = "1485060268963074049-GaOXdQp88hsU5o6pAmmapSD99BK5Zi"
accessTokenSecret = "09Pd290PSDClZaJ350fKigKhyJBfWhNlEo3kYEpBhVyNf"
bearerToken = "AAAAAAAAAAAAAAAAAAAAAJSlYQEAAAAAr0ClDhRusUwMfk5JC5N4n6HPRvE%3DsEVzWdUqW3mGF6uOCgt1umA3Dc7XOZEwA104RNBzSQSyybGAS8"
# auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth = tweepy.AppAuthHandler(consumerKey, consumerSecret)
# auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)


In [18]:
client = tweepy.Client(bearerToken)
#query, *, end_time, expansions, max_results, media_fields, next_token, place_fields, poll_fields, since_id, start_time, tweet_fields, until_id, user_fields
start_time = "2022-01-21T21:05:00Z"
end_time = "2022-01-21T23:05:00Z"
for response in tweepy.Paginator(client.search_all_tweets,"Bitcoin",
                                end_time=end_time,start_time=start_time,max_results=10):
    print(response.meta)

Forbidden: ignored

In [11]:
# give tweets related to keyword
print(consumerKey)
def get_tweets(keyword):
    t = tweepy.Cursor(api.search, q=keyword).items(10)
    print(t)
    tweet_lst = []
    for item in t:
        tweet_lst.append(item.text)
    return tweet_lst

get_tweets('bitcoin')



2xahjEG9TT3v3lV2kFCdFAEjt


AttributeError: ignored

In [20]:
import requests
h = {'Authorization': 'Bearer {}'.format(bearerToken)}
r = requests.get('https://api.twitter.com/2/tweets/search/recent?query=from:twitterdev',headers = h)
print(r.text)

{"data":[{"id":"1484247876817534989","text":"Thanks for your feedback on the v2 Retweets and Likes endpoints. We've heard you. Starting today, you can retrieve the complete list of accounts that have Liked or Retweeted a Tweet. Don't forget to tell us what you'll build! #BuildWhatsNext \n\nhttps://t.co/z5RhIVxJFK"},{"id":"1484214440241283072","text":"Join us on Spaces Jan 26th at 9am PST / 12pm EST to meet and chat with the Developer Advocacy team members.\n\nhttps://t.co/iV0s6hgbN6"}],"meta":{"newest_id":"1484247876817534989","oldest_id":"1484214440241283072","result_count":2}}


In [9]:
def preprocess():
    tweet_lst = pd.DataFrame(tweet_lst).drop_duplicates(inplace=True)
    tweet_lst['text'] = tweet_lst[0]
    remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
    rt = lambda x: re.sub("(@[A-Za-z0–9]+)|([⁰-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
    tweet_lst['text'] = tweet_lst.text.map(remove_rt).map(rt)
    tweet_lst['text'] = tweet_lst.text.str.lower()
    return tweet_lst

4. Score tweets based on sentiment prediction

In [105]:
def pred(tweet):
    tokens = tokenizer.encode(tweet, return_tensors='pt')
    result = model(tokens)
    return np.arange(1,6) @ result.logits.softmax(dim=1).tolist()[0]

pred('This coin is a good buy, I see a lot of potential in it')

4.168526469031349

5. Handle requests for specific coins


In [ ]:
df["Score"].mean()